In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10260
2,Huelva,Confirmados PDIA 14 días,1142
3,Huelva,Tasa PDIA 14 días,"223,59582020703172"
4,Huelva,Confirmados PDIA 7 días,468
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,3
629,Municipio de Huelva sin especificar,Total Confirmados,125
630,Municipio de Huelva sin especificar,Curados,88


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10260.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2886.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1056 personas en los últimos 7 días 

Un positivo PCR cada 437 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10260.0,468.0,1142.0,510743.0,91.631212,223.595820,22.0
Huelva-Costa,5722.0,323.0,769.0,288115.0,112.108012,266.907311,16.0
Huelva (capital),2886.0,136.0,329.0,143663.0,94.665989,229.008165,10.0
Condado-Campiña,3151.0,129.0,320.0,155057.0,83.195212,206.375720,6.0
Aljaraque,390.0,22.0,56.0,21260.0,103.480715,263.405456,2.0
Almonte,434.0,2.0,10.0,24191.0,8.267538,41.337688,1.0
Bollullos Par del Condado,235.0,5.0,10.0,14272.0,35.033632,70.067265,1.0
Cartaya,348.0,32.0,71.0,19974.0,160.208271,355.462101,1.0
Lepe,903.0,42.0,118.0,27431.0,153.111443,430.170245,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,83.0,5.0,24.0,3371.0,148.323939,711.954910,1.0
Palma del Condado (La),478.0,32.0,70.0,10761.0,297.370133,650.497166,1.0
Isla Cristina,376.0,70.0,114.0,21264.0,329.194883,536.117381,0.0
Sanlúcar de Guadiana,4.0,1.0,2.0,409.0,244.498778,488.997555,0.0
San Bartolomé de la Torre,44.0,6.0,18.0,3736.0,160.599572,481.798715,0.0
Lepe,903.0,42.0,118.0,27431.0,153.111443,430.170245,1.0
Villalba del Alcor,102.0,6.0,14.0,3338.0,179.748352,419.412822,0.0
Cartaya,348.0,32.0,71.0,19974.0,160.208271,355.462101,1.0
Cortelazor,2.0,1.0,1.0,299.0,334.448161,334.448161,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,317.0,6.0,46.0,20946.0,28.645087,219.612336,0.0,0.130435
Valverde del Camino,271.0,2.0,12.0,12820.0,15.600624,93.603744,0.0,0.166667
Beas,56.0,1.0,6.0,4257.0,23.490721,140.944327,0.0,0.166667
Almonte,434.0,2.0,10.0,24191.0,8.267538,41.337688,1.0,0.200000
Lucena del Puerto,83.0,5.0,24.0,3371.0,148.323939,711.954910,1.0,0.208333
Rociana del Condado,115.0,4.0,16.0,7855.0,50.922979,203.691916,1.0,0.250000
Puebla de Guzmán,24.0,1.0,4.0,3073.0,32.541490,130.165962,1.0,0.250000
Villanueva de los Castillejos,58.0,2.0,8.0,2820.0,70.921986,283.687943,0.0,0.250000
Sierra de Huelva-Andévalo Central,1263.0,13.0,47.0,67571.0,19.239023,69.556467,0.0,0.276596
